In [0]:
dbutils.fs.mount(source='wasbs://sales@mayastorageaccco.blob.core.windows.net', mount_point='/mnt/salesp', extra_configs={'fs.azure.account.key.mayastorageaccco.blob.core.windows.net': '0rXKU6yI9OHGzXaczcU9OoTfiHPCIqxih7SpK46YkqZLv/QyUDmwc743J4jU5U2XTc351GeLI/3G+AStTiyHOw=='})

In [0]:
%fs

ls /mnt/salesp

path,name,size,modificationTime
dbfs:/mnt/salesp/discarding/,discarding/,0,0
dbfs:/mnt/salesp/landing/,landing/,0,0
dbfs:/mnt/salesp/staging/,staging/,0,0


In [0]:
%fs

ls /mnt/salesp/landing

path,name,size,modificationTime
dbfs:/mnt/salesp/landing/orders.csv,orders.csv,203,1723973475000


In [0]:
ordersDf=spark.read.csv('/mnt/salesp/landing/orders.csv', inferSchema=True,header=True)
display(ordersDf)

order_id,order_date,customer_id,order_status
1111111,00:00.0,11599,CLOSED
2222222,00:00.0,256,PENDING_PAYMENT
3333333,00:00.0,12111,COMPLETE
4444444,00:00.0,8827,CLOSED
555555,00:00.0,11599,CLOSED


In [0]:
errorFlg=False

ordersCount= ordersDf.count()
print(ordersCount)

5


In [0]:
distinctOrderCount=ordersDf.select('order_id').distinct().count()
print(distinctOrderCount)

if ordersCount != distinctOrderCount:
    errorFlg= True

    if errorFlg:

        dbutils.fs.mv('/mnt/salesp/landing/orders.csv', 'mnt/salesp/discarding')
        dbutils.notebook.exit('{"errorFlg" : "true", "errorMsg","Orderid is repeated"}')

ordersDf.createOrReplaceTempView("orders")

5


In [0]:
dbServer ='mayadevser'
dbPort='1433'
dbName='mayadbsqlp'
dbUser='mayaadmin'
dbPassword='sqldbpassword'
databricksScope='secret-scope-sales-project'

In [0]:
connectionUrl=(
'jdbc:sqlserver://{}.database.windows.net:{};database={};user={};'.format(dbServer,dbPort,dbName,dbUser))
dbPassword=dbutils.secrets.get(scope=databricksScope, key='sqldbpassword')
connectionProperties={
   'password': dbPassword,
    'driver': 'com.microsoft.sqlserver.jdbc.SQLServerDriver'
}


In [0]:
validStatusDf=spark.read.jdbc(url=connectionUrl,table='dbo.valid_order_status',properties=connectionProperties)

In [0]:
display(validStatusDf)

status_name
ON HOLD
PAYMENT_REVIEW
PROCESSING
CLOSED
SUSPECTED_FRAUD
COMPLETE
PENDING
CANCELLED
PENDING_PAYMENT


In [0]:
validStatusDf.createOrReplaceTempView("valid_status")

In [0]:
invalidRowsDf=spark.sql("select * from orders where order_status not in(select * from valid_status)")

In [0]:
display(invalidRowsDf)

order_id,order_date,customer_id,order_status


In [0]:
if invalidRowsDf.count() > 0 :
    errorFlg=True
if errorFlg:
   if errorFlg:
    dbutils.fs.mv('/mnt/salesp/landing/orders.csv', 'mnt/salesp/discarding')
    dbutils.notebook.exit('{"errorFlg" : "true", "errorMsg": "invalid order status found"}')
else:
        dbutils.fs.mv('/mnt/salesp/landing/orders.csv', 'mnt/salesp/staging')
        dbutils.notebook.exit('{"errorFlg" : "false", "errorMsg","all good"}')

